In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

In [10]:
print(list(base_model.children())[-1])


Linear(in_features=2048, out_features=1000, bias=True)


In [11]:
num_cls = 30

In [44]:
class PretextModel(nn.Module):
    def __init__(self, num_cls):
        super(PretextModel, self).__init__()
        base_model = models.resnet50(pretrained=False)
        self.base_model = nn.Sequential(*list(base_model.children())[:-1])
        self.classifier = nn.Linear(2048, num_cls)
    
    def forward(self, x):
        x = self.base_model(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    

In [45]:
pretext_model = PretextModel(30)


In [48]:
inp = torch.randn(5, 3, 245,245)
out = pretext_model(inp)
print(out.shape)

torch.Size([5, 30])
